In [1]:
import subprocess

In [15]:
!clingo q0.db datalog_query0.lp

clingo version 5.6.2
Reading from q0.db ...
Solving...
Answer: 1
satisfied
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.127s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.141s


In [7]:
!clingo q0.db gac_query0.lp

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [53]:
!clingo query1.db datalog_query1.lp

clingo version 5.6.2
Reading from query1.db ...
Solving...
Answer: 1
satisfied
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.007s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


In [54]:
!clingo query1.db gac_query1.lp

clingo version 5.6.2
Reading from query1.db ...
Solving...
Answer: 1
p(1,1,1,1) p(2,2,3,2) vrow(100,r,x,x) vrow(100,r,y,y) vrow(100,r,z,z) vrow(100,r,w,w) kd(1,r,x) kd(2,r,y) kd(3,r,z)
SATISFIABLE

Models       : 1
Calls        : 1
Time         : 0.001s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.000s


In [16]:
def generate_database(i,q,c):
    j=i+1000
    nbr=0
    db=""
    while(nbr<10000):
        if(c):
            if(q==0):
                db=db+f'p({j},{(j+((j+1)%2))},{(j+(j%2))*2}). s({j+((j+1)%2)},0). r({(j+(j%2))*2},{j+3},c). '
            else:
                db=db+f'p({j},{(j+((j+1)%2))},{(j+(j%2))*2},{(j+(j%2))*2}).'
        else:
            break
        j=j+1
        nbr=nbr+1
    return db

In [ ]:
f = open("q0.db", "w")

i=0
result_datalog=[]
result_gac=[]
while(i<10):
    line=generate_database(i,0,True)
    f.write(line)
    
    result = subprocess.run(['clingo','q0.db',"datalog_query0.lp"], stdout=subprocess.PIPE, universal_newlines=True)
    tmp = result.stdout.split("CPU Time     : ")
    result_datalog.append(tmp[1].split("s")[0])
    if("unsatisfied" in tmp[0]):
        print("Unsatisfied detected")
        break

    result = subprocess.run(['clingo','q0.db',"gac_query0.lp"], stdout=subprocess.PIPE, universal_newlines=True)
    tmp = result.stdout.split("CPU Time     : ")
    result_gac.append(tmp[1].split("s")[0])
    if("unsatisfied" in tmp[0]):
        print("Unsatisfied detected")
        break
        
        
    i=i+1


f.close()

In [22]:
print(result_datalog)
print(result_gac)

['0.156', '0.281', '0.500', '0.516', '0.391', '0.781', '0.922', '0.641', '1.156', '1.281']
['0.359', '0.578', '0.734', '0.906', '0.656', '0.734', '1.344', '0.875', '0.953', '1.781']


In [ ]:
f = open("q0_inconsistent.db", "w")

i=0
result_datalog=[]
result_gac=[]
while(i<10):
    line=generate_database(i,0,True)
    f.write(line)
    
    result = subprocess.run(['clingo','q0.db',"datalog_query0.lp"], stdout=subprocess.PIPE, universal_newlines=True)
    result_datalog.append(result.stdout.split("CPU Time     : ")[1].split("s")[0])

    result = subprocess.run(['clingo','q0.db',"gac_query0.lp"], stdout=subprocess.PIPE, universal_newlines=True)
    result_gac.append(result.stdout.split("CPU Time     : ")[1].split("s")[0])
    
    i=i+1


f.close()

0
1
1
